In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import glm_model as gm
from glm_utils import *
import tensorflow as tf

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#finally get rid of the ugly ass allen code. We import the pickled design 
#matrix stimulus, which is PCA projection of the images, and the downsampled
#traces. 

import pickle

pkl_file = open('./boc/formatted/LMcelldata')
data = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('./boc/formatted/LMstimdata')
stim = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('./boc/formatted/runspeed')
run_speed = pickle.load(pkl_file)
pkl_file.close()

In [3]:
#get the number of features, and the number of training examples. 
l, n =  stim.shape

In [ ]:
from statsmodels.tsa.tsatools import lagmat
from sklearn.cross_validation import train_test_split
from scipy.stats import linregress
from itertools import product
import pyprind

import sys

n_pcs = range(10, 110, 5)
non_lin = [0]

alpha = np.logspace(-1,1, 10) 
nlag = 2

#empty dictionary of the best model we've fit for each cell
best_model = {k: (-np.inf, None) for k in data.keys()}

#make a dictionary of dictionaries to store the results
scores_dict = {k: {} for k in data.keys()}

bar = pyprind.ProgBar(len(alpha)*len(non_lin))

for (a, n) in product(alpha, n_pcs):
    bar.update()
    
    #lagged_stim = lagmat(stim[:, 0:n], maxlag = nlag, trim = None)[1:-1]       
    #stim_array = np.concatenate((stim[:, 0:n], run_speed[:, np.newaxis]), axis = 1)
    
    
    for c in data.keys():
        X_train, X_test, y_train, y_test = train_test_split(stim[:, 0:n], np.array(data[c])) 
        
        y_train[y_train < 0], y_test[y_test <0] = 0, 0
        
        weight_init = np.linalg.pinv(X_train).dot(y_train)[:, np.newaxis]
          
    
        if l == 0:
            nonlin = tf.exp
        if l == 1:
            nonlin = tf.nn.sigmoid
        if l == 2:
            nonlin = tf.nn.relu

        
        model = gm.exponential_GLM(weight_init, non_lin = nonlin, verbose = False, alpha = 0, reg = 'l2', 
                                lr = 1e-3, train_params = False, scale_init = 1, bias_init = 0)
        
        loss, cv = model.fit(X_train, y_train[:, np.newaxis], X_test, y_test[:, np.newaxis], 
                             max_iters = 60) 

        
        w, o, s  = model.get_params()

    
        _, _, r2_test,_, _ = linregress(np.squeeze(model.predict_trace(X_test)), np.squeeze(y_test))
        _, _, r2_train,_, _ = linregress(np.squeeze(model.predict_trace(X_train)), np.squeeze(y_train))

        scores_dict[c][(l, n)] = (r2_test, r2_train) 

        if r2_test > best_model[c][0]:
            best_model[c] = (r2_test, (l, n))
            print "new best model for cell ", c, ". r2 test: ", r2_test


0%      100%
[#         ] | ETA: 00:00:00

In [5]:
for c in data.keys():
    
    if best_model[c][1] != None:
        (v, (l, n)) = best_model[c]
    
        print 'explained var: ', round(v, 3), 'non_lin:', l, 'n_components', n
        
    print best_model[c]

explained var:  0.033 non_lin: 0 n_components 11
(0.033481352484106865, (0, 11))
explained var:  0.029 non_lin: 0 n_components 11
(0.0291953538059119, (0, 11))
explained var:  0.038 non_lin: 0 n_components 1
(0.038053710902818907, (0, 1))
explained var:  0.02 non_lin: 0 n_components 31
(0.020302991123940045, (0, 31))
explained var:  0.039 non_lin: 0 n_components 21
(0.038589112636662745, (0, 21))
explained var:  0.047 non_lin: 0 n_components 1
(0.047027122449560151, (0, 1))
explained var:  0.03 non_lin: 0 n_components 1
(0.029830916758050888, (0, 1))
explained var:  0.029 non_lin: 0 n_components 71
(0.029482966787885187, (0, 71))
explained var:  0.027 non_lin: 0 n_components 11
(0.027403796402451115, (0, 11))
explained var:  0.012 non_lin: 0 n_components 71
(0.012212338928765543, (0, 71))
explained var:  0.071 non_lin: 0 n_components 1
(0.071203885108500731, (0, 1))
explained var:  0.008 non_lin: 0 n_components 11
(0.0081442650532704292, (0, 11))
explained var:  0.046 non_lin: 0 n_comp

In [7]:
pkl_file = open('./boc/formatted/LNE_exp', 'wb')

pickle.dump((best_model, scores_dict), pkl_file)
pkl_file.close()